In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


payload = {
  "api_key": "jorXNk0XeOjcNxNdmsBHa9YXUKSwnSgMFChONEcLZh4UVsc12swTZjUE2rgfKdEgb1L7KEdEs84IhmobWb",
  "url": "https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B%5D=ti&showCol%5B%5D=fm&showCol%5B%5D=oa&showCol%5B%5D=at&showCol%5B%5D=md&showCol%5B%5D=df&showCol%5B%5D=cw&showCol%5B%5D=ps",
}

response = requests.get("https://scraping.narf.ai/api/v1/", params=payload)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:

base_url = "https://sofifa.com"


select_season = soup.find('select', {'id': 'select-version'})

for season_option in select_season.find_all('option'):
    season_url = base_url + season_option.get('value')
    season_name = season_option.get_text(strip=True)
    print(f"Scraping season: {season_name}")

    payload = {
        "api_key": "jorXNk0XeOjcNxNdmsBHa9YXUKSwnSgMFChONEcLZh4UVsc12swTZjUE2rgfKdEgb1L7KEdEs84IhmobWb",
        "url": season_url
    }
    season_resp = requests.get("https://scraping.narf.ai/api/v1/", params=payload)
    season_soup = BeautifulSoup(season_resp.text, 'html.parser')

    # find roster dropdown for this season
    select_roster = season_soup.find('select', {'name': 'roster'})
    if not select_roster:
        print(f"No roster dropdown for season {season_name}")
        continue
    
    for roster_option in select_roster.find_all('option'):
            relative_url = roster_option.get('value')
            full_url = base_url + relative_url
            date = roster_option.get_text(strip=True)
            safe_date = date.replace("/", "-").replace(":", "-").strip()
            file_path = f'../../data_files/scraped_raw/teams/teams_{safe_date}.csv'
            
            try:
                payload = {
                    "api_key": "jorXNk0XeOjcNxNdmsBHa9YXUKSwnSgMFChONEcLZh4UVsc12swTZjUE2rgfKdEgb1L7KEdEs84IhmobWb",
                    "url": full_url
                }
                response = requests.get("https://scraping.narf.ai/api/v1/", params=payload)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                table = soup.find('table')
                if table is None:
                    print(f'No table found for: {full_url}')
                    continue
    

                rows = table.find_all('tr')
                if not rows or not rows[0].find_all('th'):
                    print(f'no headers found: {full_url}')
                    continue

                headers = [th.get_text(strip=True) for th in rows[0].find_all('th')]
                data = []
            
            
                for row in rows[1:]:
                    cols = [td.get_text(strip=True) for td in row.find_all('td')]
                    if cols:
                        data.append(cols)

                df = pd.DataFrame(data, columns=headers)

                df.to_csv(file_path, index=False) 
            except Exception as e:
                print(f'failed to scrape {full_url}')
                continue

Scraping season: FC 26
Scraping season: FC 25
Scraping season: FC 24
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=202400&set=true
Scraping season: FIFA 23
No table found for: https://sofifa.com/teams?type=all&lg%5B0%5D=39&showCol%5B0%5D=ti&showCol%5B1%5D=fm&showCol%5B2%5D=oa&showCol%5B3%5D=at&showCol%5B4%5D=md&showCol%5B5%5D=df&showCol%5B6%5D=cw&showCol%5B7%5D=ps&r=202200&set=true
Scraping season: FIFA 22
Scraping season: FIFA 21
Scraping season: FIFA 20
Scraping season: FIFA 19
Scraping season: FIFA 18
Scraping season: FIFA 17
Scraping season: FIFA 16
Scraping season: FIFA 15
Scraping season: FIFA 14
Scraping season: FIFA 13
Scraping season: FIFA 12
Scraping season: FIFA 11
Scraping season: FIFA 10
Scraping season: FIFA 09
Scraping season: FIFA 08
Scraping season: FIFA 07
